# Tries

In [1]:
# end-of-string sentinel
END_OF_STRING = object()
    

class TrieNode:
    
    def __init__(self):
        self.children = {}
        
    def insert(self, s, start=0, stop=None):
        """Insert s[start:stop] into the trie."""
        if stop is None:
            stop = len(s)
            
        if start >= stop:
            self.children[END_OF_STRING] = TrieNode()
            return
        
        if s[start] not in self.children:
            self.children[s[start]] = TrieNode()
            
        child = self.children[s[start]]
        child.insert(s, start + 1, stop)
        
    def walk(self, s, start=0, stop=None):
        """Walk the trie following s[start:stop].
        Raises ValueError if falls off tree.
        Returns last node encountered otherwise."""
        if stop is None:
            stop = len(s)
            
        if start >= stop:
            return self
        
        if s[start] not in self.children:
            raise ValueError('Fell off tree.')
        else:
            child = self.children[s[start]]
            return child.walk(s, start + 1, stop)
    
    def membership_query(self, s, start=0, stop=None):
        """Determine True/False if s[start:stop] is in trie."""
        try:
            node = self.walk(s, start, stop)
        except ValueError:
            return False
        
        return END_OF_STRING in node.children
        
    def produce(self, prefix=''):
        """Generate the words in the trie."""
        for letter, child in self.children.items():
            if letter is END_OF_STRING:
                yield prefix
            else:
                yield from child.produce(prefix + letter)
                
    def complete(self, prefix):
        try:
            node = self.walk(prefix)
        except ValueError:
            return []
        return list(node.produce())

## Small Example

Here is a small example of a trie containing just four words.

In [2]:
root = TrieNode()
root.insert('testing')
root.insert('and')
root.insert('this')
root.insert('that')

In [3]:
root.complete('th')

['is', 'at']

## All Words in *Ulysses*

Let's read all of the words in Joyce's *Ulysses* into a trie.

In [4]:
def clean_up_word(s):
    return s.strip('.,').lower()

ulysses_words = set()
for line in open('ulysses.txt'):
    for word in line.split():
        ulysses_words.add(clean_up_word(word))

In [5]:
ulysses_trie = TrieNode()
for word in ulysses_words:
    ulysses_trie.insert(word)

We'll compare the trie against a brute force prefix search:

In [6]:
def brute_force_prefix_search(words, prefix):
    matches = []
    for word in words:
        if word.startswith(prefix):
            matches.append(word)
    return matches

How long does it take to find all words in the book that start with `'thu'`, using the brute force search?

In [7]:
%%timeit
brute_force_prefix_search(ulysses_words, 'thu')

4.17 ms ± 34.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Let's compare this to the trie.

In [8]:
%%timeit
ulysses_trie.complete('thu')

54.2 µs ± 885 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


On my machine, I find that the brute force search through a set takes around 4 milliseconds, while the trie takes 52 microseconds. That's a pretty big speedup!

What about a membership query?

In [9]:
%%timeit
'thunderation!' in ulysses_words

39.2 ns ± 0.724 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [10]:
%%timeit
ulysses_trie.membership_query('thunderation!')

5.46 µs ± 61.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


The trie is *considerably slower*. Why is this? Hash tables and tries both take $\Theta(|p|)$ (expected) time to perform a membership query. A query in a trie simply walks down $|p|$ nodes of the tree. A hash table, on the other hand, requires hashing the query string in $\Theta(|p|)$, then must loop through a bucket, performing a string equality check on each entry in the bucket. So the total time taken by a hash table is $\Theta(|p|) + c \cdot O(|p|)$, where $c$ is the number of elements in the bucket which appear before the query string.

In some sense, the hash table is doing a lot more work and we'd therefore expect it to take longer. But there are plenty of factors which mean that the hash table is actually faster here. For one, Python hash tables are *super* optimized. The $\Theta(|p|)$ work needed to hash the string is done in C, while the $\Theta(|p|)$ work done by the trie is in a pure Python loop and involves a lot of indirection. If we wanted to make trie membership query more competitive, we'd need to implement our trie in a lower-level language (C, Rust, etc.). The hash table is plenty fast.

## Memory Usage

It might seem that a trie will result in memory savings. For instance, suppose we stored the strings `"AAAG"` and `"AAAC"` in a trie and in a hash table. On one hand, the trie is able to "de-duplicate" the common prefixes because it doesn't store the strings themselves -- they are encoded by their paths in the trie. A hash table, on the other hand, stores both strings in their entirety, including the duplicate prefixes.

In practice, however, the memory savings are hard to realize due to the fact that the trie must maintain a hash table for *every* node. Hash tables (especially in Python) have considerable memory overhead.

This short experiment will demonstrate the problem. First, let's consider the total number of characters in all words in the *Ulysses* data set.

In [11]:
sum(len(word) for word in ulysses_words)

271200

If we use a hash table to store the words directly, this is the number of individual characters that it will need to allocate memory for. On the other hand, let's count the number of nodes in the trie representing the same collection of words -- this will be close to the number of characters stored in the trie, since each edge represents a character:

In [12]:
def number_of_nodes(root):
    return 1 + sum(number_of_nodes(child) for child in root.children.values())

In [13]:
number_of_nodes(ulysses_trie)

143192

That's about half of what the hash table uses! This number depends on the data itself and how many common prefixes there are. In the "best case" for a trie, the strings all have long, common prefixes, like:

- `"AAAAAAAAAAAGATTACA"`
- `"AAAAAAAAAAACTATTAC"`
- `"AAAAAAAAAAAGGGATAC"`

Here, the trie "compresses" the common prefixes into a common path, and stores the leading `A`s once each, instead of three times each. But in the hash table, each `A` is stored.

So in short, the trie is able to keep track of maybe half the number of characters the hash table approach remembers. But a trie has the additional overhead of needing to store a hash table in each node. How much memory does this use? We can use `sys.getsizeof` to get an estimate.

In [14]:
import sys
sys.getsizeof(dict())

232

It looks like an empty dictionary takes 232 bytes of memory. How does this compare to a single character?

In [15]:
sys.getsizeof('a')

50

An empty dictionary is almost 5 times as large as a character! This means that even if the `.children` dictionaries of each node were all empty, the trie would take 4-5 times as much memory as a single hash table storing all of the strings! And this doesn't even take into account the memory needed to represent the `TrieNode` object itself.

In some cases it might be possible to see memory savings with tries. For instance, if our trie stores strings over the alphabet G, A, T, C, we don't need to store a dictionary at each node; it suffices to store 4 (possibly-null) pointers to the children. Each pointer is probably a 64-bit integer, taking 32 bytes in total. But to take advantage of this, we'd need to implement our trie in a low-level language (C, C++, Rust, etc.). Python isn't built for such low-level memory manipulation.